In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('./csv_data_files/movie_rating.csv')
ratings

,critic,title,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0
5,Mick,Lady,3.0
6,Mick,Snakes,4.0
7,Mick,Just My,2.0
8,Mick,Superman,3.0
9,Mick,You Me,2.0


In [3]:
movie_titles = list(set(ratings['title']))
movie_titles

['Lady', 'Superman', 'Snakes', 'You Me', 'Just My', 'The Night']

In [4]:
critic = list(set(ratings['critic']))
critic

['Mick', 'Toby', 'Gene', 'Claudia', 'Jack', 'Lisa']

In [5]:
df = pd.crosstab(index = ratings.critic, columns = ratings.title, 
            values = ratings.rating, aggfunc = np.sum)
df

title,Just My,Lady,Snakes,Superman,The Night,You Me
critic,,,,,,
Claudia,3.0,NaN,3.5,4.0,4.5,2.5
Gene,1.5,3.0,3.5,5.0,3.0,3.5
Jack,NaN,3.0,4.0,5.0,3.0,3.5
Lisa,3.0,2.5,3.5,3.5,3.0,2.5
Mick,2.0,3.0,4.0,3.0,3.0,2.0
Toby,NaN,NaN,4.5,4.0,NaN,1.0


In [6]:
df.index

Index(['Claudia', 'Gene', 'Jack', 'Lisa', 'Mick', 'Toby'], dtype='object', name='critic')

In [7]:
import surprise

from surprise import SVD # 고유값분해(행렬을 더 낮은 차원으로 분해 but 고유한 성질은 그대로)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset, Reader # SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                             # 사용자, 영화, 평점
from surprise import accuracy # RMSE, MSE, CrossValidation(k-fold)
from surprise.model_selection import train_test_split # 훈련/검증 데이터 분류

In [8]:
reader = Reader(rating_scale=(0.5, 5.0))

In [9]:
# SVD에서 사용할 수 있는 데이터셋으로 만들어주자.
data = Dataset.load_from_df(ratings[['critic', 'title', 'rating']], reader)
data

In [10]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [11]:
# 객체 생성
svd = SVD(n_factors=50, n_epochs= 40,  random_state=0)

# 훈련
svd.fit(trainset) 

# 검증
predictions = svd.test(testset)

# 결과분석
accuracy.rmse(predictions)

RMSE: 0.8610


0.8610455895357841

In [12]:
from surprise.model_selection import cross_validate

In [13]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6350  0.5220  1.0787  0.6386  1.0710  0.7890  0.2371  
MAE (testset)     0.4914  0.4633  0.9160  0.4526  0.6588  0.5964  0.1763  
Fit time          0.01    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.63498332, 0.52202212, 1.07865484, 0.63855181, 1.07103306]),
 'test_mae': array([0.4914113 , 0.46334164, 0.91598752, 0.45257193, 0.65877644]),
 'fit_time': (0.005028963088989258,
  0.003364086151123047,
  0.003525257110595703,
  0.004836082458496094,
  0.004389047622680664),
 'test_time': (0.00014400482177734375,
  0.00014281272888183594,
  0.002477884292602539,
  0.00016808509826660156,
  0.00016999244689941406)}

In [14]:
## 내가 안본 영화 리스트 구해서, 이중에서 추천하려고 함.
def get_unseen_surprise(titles, critics, ratings):
    
    # 전체 영화id 리스트
    total_movies = movie_titles
    
    # 내가 본 영화id 리스트
    seen_movies = ratings[ratings['critic'] == critics]['title'].tolist()
    
    # 추천 대상이 되는 영화 리스트: 전체 영화 리스트 - 내가 본 영화 리스트
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    
    print("전체 영화 리스트 개수 >> ", len(total_movies))
    print("내가 본 영화 리스트 개수 >> ", len(seen_movies))
    print("내가 안 본 영화 리스트 개수 >> ", len(unseen_movies))

    return unseen_movies

In [15]:
unseen_movies = get_unseen_surprise(movie_titles, 'Toby', ratings)
unseen_movies

전체 영화 리스트 개수 >>  6
내가 본 영화 리스트 개수 >>  3
내가 안 본 영화 리스트 개수 >>  3


['Lady', 'Just My', 'The Night']

In [21]:
## 안본 영화중에서 평점 예측이 높게 나온 3개를 리스트업하는 함수
def recomm_movie_by_surprise(svd, critic, unseen_movies):
    
    # 안 본 영화리스트를 하나씩 꺼낸 다음 평점을 예측하세요
    predictions = [svd.predict(critic, title) for title in unseen_movies]
    
    # 평점이 높은 순으로 정렬하는 기준을 함수화
    def sortkey_est(one):
        return one.est
    
    # 평점이 높은 순으로 정렬해서 top10 추천
    predictions.sort(key=sortkey_est,reverse=True)
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    critics = [pred.uid for pred in predictions]
    titles = [pred.iid for pred in predictions]
    ratings = [pred.est for pred in predictions]
    preds = [(critic, title, np.round(rating, 2)) for critic, title, rating in zip(critics, titles, ratings)]
    
    return preds

In [22]:
recomm_movie_by_surprise(svd, 'Toby', unseen_movies)

[('Toby', 'The Night', 3.53),
 ('Toby', 'Lady', 3.23),
 ('Toby', 'Just My', 3.15)]